In [ ]:
!pip install POT

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 kB 8.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import gdown
import gensim.downloader as api
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
from scipy.spatial.distance import jaccard, dice, hamming
from scipy.stats import pearsonr, entropy
from sklearn.cluster import KMeans
from gensim.corpora import Dictionary
from gensim.similarities import WmdSimilarity
from gensim.models import KeyedVectors
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

url = "https://drive.google.com/uc?id=1y9qBtV71aY6OtEnhfQAxMTWXBBmdv63i"
output = "dataset.csv"
gdown.download(url, output, quiet=False, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1y9qBtV71aY6OtEnhfQAxMTWXBBmdv63i
To: /content/dataset.csv
100%|██████████| 93.0M/93.0M [00:00<00:00, 203MB/s]


'dataset.csv'

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

# Modeling

In [ ]:
input_text = "At the age of 28, I, John Smith, am an enthusiastic and highly skilled data analyst seeking to advance my career by applying for a position in the data field. With over five years of experience in data analysis and a proven track record of successfully implementing data-driven strategies, I have honed my expertise in various analytical tools and methodologies. I hold a Bachelor's degree in Computer Science from the University of California, Berkeley, and a Master's degree in Data Science from Stanford University. Throughout my career, I have demonstrated exceptional analytical capabilities, strong problem-solving skills, and a deep understanding of statistical models and machine learning algorithms. My passion for uncovering insights from complex datasets and my commitment to continuous learning make me a valuable candidate for any data-driven organization."
input_text = input_text.lower()
input_text = re.sub(r'[^\w\s]', '', input_text)
tokens = word_tokenize(input_text)
stop_words = set(stopwords.words('english'))
tokens = [word for word in tokens if word not in stop_words]
lemmatizer = WordNetLemmatizer()
tokens = [lemmatizer.lemmatize(word) for word in tokens]
input_text = ' '.join(tokens)

In [ ]:
input_text

'age 28 john smith enthusiastic highly skilled data analyst seeking advance career applying position data field five year experience data analysis proven track record successfully implementing datadriven strategy honed expertise various analytical tool methodology hold bachelor degree computer science university california berkeley master degree data science stanford university throughout career demonstrated exceptional analytical capability strong problemsolving skill deep understanding statistical model machine learning algorithm passion uncovering insight complex datasets commitment continuous learning make valuable candidate datadriven organization'

kesimpulan dari input text ini adalah sebagai berikut:

- pengalaman lima tahun dalam analisis data dan memiliki rekam jejak yang terbukti dalam menerapkan strategi berbasis data dengan sukses

- memiliki keahlian dalam berbagai alat dan metodologi analitis

- gelar sarjana di bidang Ilmu Komputer dari University of California, Berkeley, dan gelar master di bidang Ilmu Data dari Stanford University

- kemampuan analitis yang luar biasa, keterampilan pemecahan masalah yang kuat, pemahaman mendalam tentang model statistik dan algoritma pembelajaran mesin, serta hasrat untuk mengungkap wawasan dari dataset yang kompleks.

kemungkinan tu di bidang data science, analisis, ML dan AI

### Parameter Tunning

#### model pre-trained "glove-wiki-gigaword-300"

(ini yang digunakan pada model asli tanpa di tunning lebih lanjut)

Menggunakan model yang pre-training "glove-wiki-gigaword-300"

valid_words = [word for word in tokens if word in model]

In [ ]:
glove_vectors = api.load("glove-wiki-gigaword-300")

[==================================================] 100.0% 376.1/376.1MB downloaded


In [ ]:
def get_average_glove(tokens, model, num_features):
    valid_words = [word for word in tokens if word in model]
    if not valid_words:
        return np.zeros(num_features)
    return np.mean([model[word] for word in valid_words], axis=0)

dataset = pd.read_csv(output)

corpus_tokens = [word_tokenize(re.sub(r'[^\w\s]', '', doc.lower())) for doc in dataset["deskripsi_keterampilan_final"]]
corpus_tokens = [[lemmatizer.lemmatize(word) for word in tokens if word not in stop_words] for tokens in corpus_tokens]
num_features = 300
corpus_embeddings = np.array([get_average_glove(tokens, glove_vectors, num_features) for tokens in corpus_tokens])

input_tokens = word_tokenize(input_text)
input_embedding = get_average_glove(input_tokens, glove_vectors, num_features)

In [ ]:
cosine_similarities = cosine_similarity([input_embedding], corpus_embeddings).flatten()
dataset["cosine_similarity"] = cosine_similarities

In [ ]:
pd.set_option("display.max_colwidth",None)

In [ ]:
top_similar = dataset.sort_values(by='cosine_similarity', ascending=False).head(20)
top_similar[["skill","job title","deskripsi_keterampilan_final","cosine_similarity"]]

skill  \
13111  Information Technology   
9203                    Other   
22397                Research   
19613                 Analyst   
29493               Education   
4277                  Science   
4486                  Science   
19252  Information Technology   
1784       Project Management   
19401             Engineering   
1142       Project Management   
5171                  Analyst   
6180               Consulting   
23306             Engineering   
7319              Engineering   
28867             Engineering   
23588                Research   
30445             Engineering   
4711         Customer Service   
30759  Information Technology   

                                                                       job title  \
13111           data science artificial intelligence machine learning internship   
9203   senior bioinformatics research scientist bioinformatic research scientist   
22397           group summer research program intern advanced concept technology   
19613                                                             data scientist   
29493                                    researcher doctor of philosophy student   
4277                                                       senior data scientist   
4486                                                       senior data scientist   
19252                                                             data scientist   
1784                                                              senior manager   
19401                                     research development civil engineering   
1142                                                       data scientist remote   
5171                                        senior assessment evaluation analyst   
6180                                       environmental engineer data scientist   
23306                                                          software engineer   
7319      director office of business units data science artificial intelligence   
28867                                             computer vision data scientist   
23588                          clinical data manager aggregate safety specialist   
30445                                                             data scientist   
4711                                                       senior data scientist   
30759                                                     machine learn engineer   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

top 20 kisaran 0.921 - 0.935

analisis kecocokan inputan dengan yang nilai cosin similarity tertinggi:

Kecocokan:

Pengalaman dan Keahlian dalam Analisis Data dan Pembelajaran Mesin:

Perusahaan: "familiarity programming language python", "basic understanding sql", "data analysis", "machine learning", "statistical modeling".

John Smith: "five year experience data analysis", "highly skilled data analyst", "expertise various analytical tool methodology", "deep understanding statistical model machine learning algorithm".

Cocok: John Smith memiliki pengalaman dan keahlian yang relevan dalam analisis data, pembelajaran mesin, Python, dan pemodelan statistik.

Kemampuan Pemecahan Masalah dan Analitis:

Perusahaan: "analytical individual", "identify interpret domain heavily nuanced data", "work multiple project meet quality standard deadline".

John Smith: "strong problem-solving skill", "demonstrated exceptional analytical capability".

Cocok: John Smith menunjukkan kemampuan pemecahan masalah dan analitis yang kuat, sesuai dengan kebutuhan perusahaan.

Pendidikan:

Perusahaan: "undergraduate student pursue degree computer science computer engineering related technical field".
John Smith: "bachelor degree computer science university california berkeley", "master degree data science stanford university".
Cocok: John Smith memiliki latar belakang pendidikan yang sangat kuat di bidang Ilmu Komputer dan Ilmu Data, yang bahkan lebih tinggi dari yang diminta.

Ketidakcocokan:

Status Magang:

Perusahaan: "internship unpaid", "location onsite san jose ca", "undergraduate student".
John Smith: Mencari "advance career", memiliki "five year experience".
Tidak Cocok: John Smith mencari posisi karier yang lebih maju dan sudah memiliki pengalaman kerja yang signifikan. Posisi ini adalah magang yang tidak dibayar dan ditujukan untuk mahasiswa sarjana.

Kesimpulan:

John Smith memiliki banyak kualifikasi yang sesuai dengan beberapa persyaratan perusahaan, terutama dalam hal analisis data, pembelajaran mesin, dan pemodelan statistik. Namun, posisi ini adalah magang yang tidak dibayar dan ditujukan untuk mahasiswa sarjana, sementara John Smith mencari peluang karier yang lebih maju dan sudah memiliki pengalaman kerja yang signifikan.

#### model pre-trained glove-wiki-gigaword-50

In [ ]:
glove_vectors2 = api.load("glove-wiki-gigaword-50")

[==================================================] 100.0% 66.0/66.0MB downloaded


In [ ]:
def get_average_glove(tokens, model, num_features):
    valid_words = [word for word in tokens if word in model]
    if not valid_words:
        return np.zeros(num_features)
    return np.mean([model[word] for word in valid_words], axis=0)

dataset = pd.read_csv(output)

corpus_tokens = [word_tokenize(re.sub(r'[^\w\s]', '', doc.lower())) for doc in dataset["deskripsi_keterampilan_final"]]
corpus_tokens = [[lemmatizer.lemmatize(word) for word in tokens if word not in stop_words] for tokens in corpus_tokens]
num_features = 50
corpus_embeddings = np.array([get_average_glove(tokens, glove_vectors2, num_features) for tokens in corpus_tokens])

input_tokens = word_tokenize(input_text)
input_embedding = get_average_glove(input_tokens, glove_vectors2, num_features)

In [ ]:
cosine_similarities = cosine_similarity([input_embedding], corpus_embeddings).flatten()
dataset["cosine_similarity"] = cosine_similarities

In [ ]:
top_similar = dataset.sort_values(by='cosine_similarity', ascending=False).head(20)
top_similar[["skill","job title","deskripsi_keterampilan_final","cosine_similarity"]]

,skill,job title,deskripsi_keterampilan_final,cosine_similarity
19401,Engineering,research development civil engineering,job detail qualification strong knowledge experience gi geoscience geology cfd computational fluid dynamic proficiency program language python rr gi familiarity research mathematics research physical science atmospheric science experience high performance computing hpc parallel program understanding machine learn technique data analysis prediction excellent problem solve skill ability work large complex datasets strong analytical numerical computation skill ability collaborate effectively interdisciplinary team stakeholder excellent write verbal communication skill qualification write technical report journal article disseminate knowledge gain corp engineer project collaborate engineer scientist execution research reimbursable project determine exist future potential coastal engineering navigation research need foster collaboration engineer research development center academia write proposal document obtain research reimbursable fund assemble organize data narrative form presentation publication deliver research product report software analysis database u citizenship require unofficial job post,0.972690
22397,Research,group summer research program intern advanced concept technology,advanced concept technology group look exceptional engineer scientist mathematician excite work challenge problem join team group develop technology protect deployed sailor soldier asset advanced missile threat enable operation rapidly evolve complex electronic warfare ew environment group research development activity include concept design advance algorithm development software hardware prototyping experimental field testing data analysis team focus create new capability rapid prototyping software hardware particular interest group development advanced electronic warfare system prototype advance sensor prototype novel signal processing technique resource allocation mathematical optimization approach artificial intelligence enable development next generation system fleet surface asset defense due wide ranging nature research dynamic group comprise diverse team scientist engineer expertise physic mathematics computer science various engineering background group value inclusiveness foster mentor level promote critical innovative thinking address need nation candidate part highly collaborative team thrive solve complex challenge foster innovation support career growth dedicate make positive impact nation job description summer research program intern need investigate design algorithm area classical machine learning bayesian inference work involve explore implementing compare performance use data simulation well measure data candidate comfortable read academic paper turn material high quality software tool current enrollment program working towards b phd degree mathematics computer science electrical engineering demonstrate effective organizational skill ability work independently team environment demonstrate strong write oral communication skills strong foundation probability linear algebra work knowledge statistical inference digital signal processing information theory prefer python julia programming experience strongly desire select candidate subject pre employment background investigation must able obtain maintain secret level dod security clearance mit lincoln laboratory equal employment opportunity eeo employer qualify applicant receive consideration employment discriminate basis race color religion sex sexual orientation gender identity national origin age veteran status disability status genetic information u citizenship require requisition id,0.972441
5703,Engineering,engineer,willing work entry level bachelor degree electrical mandatory bachelor degree electrical engineering electrical engineering technology equivalent military experience associate degree year experience service critical power consider high school diploma 

top 20 besar kisaran nilai 0,966 - 0,972

analisis kecocokan inputan dengan yang nilai cosin similarity tertinggi:


Kecocokan:

Pengalaman dan Keahlian dalam Analisis Data:

Perusahaan: "proficiency program language python", "experience high performance computing (hpc)", "understanding machine learn technique", "data analysis prediction".

John Smith: "highly skilled data analyst", "five year experience data analysis", "expertise various analytical tool methodology", "strong understanding statistical model machine learning algorithm".

Cocok: John Smith memiliki pengalaman dan keahlian yang relevan dalam analisis data, bahasa pemrograman Python, dan pembelajaran mesin.

Kemampuan Pemecahan Masalah dan Analitis:

Perusahaan: "excellent problem solve skill", "strong analytical numerical computation skill".

John Smith: "strong problem-solving skill", "demonstrated exceptional analytical capability".

Cocok: John Smith menunjukkan kemampuan pemecahan masalah dan analitis yang kuat, sesuai dengan kebutuhan perusahaan.

Pendidikan:

Perusahaan: Tidak secara spesifik menyebutkan pendidikan, tetapi kualifikasi terkait penelitian ilmiah mengindikasikan kebutuhan pendidikan tinggi.

John Smith: "bachelor degree computer science university california berkeley", "master degree data science stanford university".

Cocok: John Smith memiliki latar belakang pendidikan yang kuat dari institusi ternama, sesuai dengan harapan perusahaan.

Ketidakcocokan:

Geoscience dan CFD (Computational Fluid Dynamics):

Perusahaan: "strong knowledge experience gi geoscience geology cfd computational fluid dynamic".

John Smith: Tidak ada indikasi pengalaman atau pengetahuan dalam geoscience, geology, atau CFD.

Tidak Cocok: John Smith tidak menunjukkan keahlian atau pengalaman dalam bidang geoscience, geology, atau CFD.

Penulisan dan Komunikasi Teknis:

Perusahaan: "excellent write verbal communication skill", "qualification write technical report journal article".

John Smith: Meskipun ada indikasi kemampuan komunikasi, tidak ada bukti spesifik tentang penulisan laporan teknis atau artikel jurnal.

Tidak Cocok: Tidak ada informasi yang menunjukkan bahwa John Smith memiliki pengalaman khusus dalam menulis laporan teknis atau artikel jurnal.

Kesimpulan:

- John Smith memiliki banyak kualifikasi yang sesuai dengan beberapa persyaratan perusahaan, terutama dalam hal analisis data, pembelajaran mesin, dan kemampuan analitis. Namun, ada beberapa aspek penting yang tidak cocok, seperti keahlian dalam geoscience, geology, dan CFD, serta penulisan laporan teknis.

- Secara keseluruhan, John Smith bisa menjadi kandidat yang kuat untuk posisi yang lebih fokus pada analisis data dan pembelajaran mesin, tetapi mungkin kurang cocok untuk peran yang memerlukan pengetahuan mendalam dalam geoscience dan CFD.

Perbadingan model:

Model "glove-wiki-gigaword-300" dengan skor tertinggi 0.935 dan model "glove-wiki-gigaword-50" dengan skor tertinggi 0.972

Modeling "glove-wiki-gigaword-300"dengan skor tertinggi 0.935(Agile Data Pro):

Pengalaman dan Keahlian dalam Analisis Data dan Pembelajaran Mesin:

Perusahaan: "data analysis", "machine learning", "programming language python", "natural language processing nlp".

John Smith: "five year experience data analysis", "highly skilled data analyst", "deep understanding statistical model machine learning algorithm".

Cocok: John Smith memiliki pengalaman dan keahlian yang relevan dalam analisis data, pembelajaran mesin, Python, dan pemodelan statistik.

Pendidikan:

Perusahaan: "undergraduate student pursue degree computer science computer engineering related technical field".

John Smith: "bachelor degree computer science university california berkeley", "master degree data science stanford university".

Cocok: John Smith memiliki latar belakang pendidikan yang sangat kuat di bidang Ilmu Komputer dan Ilmu Data, yang bahkan lebih tinggi dari yang diminta.

Status Magang:

Perusahaan: "unpaid internship", "location onsite san jose ca".
John Smith: Mencari "advance career", memiliki "five year experience".

Tidak Cocok: John Smith mencari posisi karier yang lebih maju dan sudah memiliki pengalaman kerja yang signifikan. Posisi ini adalah magang yang tidak dibayar dan ditujukan untuk mahasiswa sarjana.

Modeling Kedua "glove-wiki-gigaword-50" dengan skor tertinggi 0.972(GI Geoscience):

Pengalaman dalam Geoscience dan CFD (Computational Fluid Dynamics):

Perusahaan: "strong knowledge experience gi geoscience geology cfd computational fluid dynamic".

John Smith: Tidak ada indikasi pengalaman atau pengetahuan dalam geoscience, geology, atau CFD.

Tidak Cocok: John Smith tidak menunjukkan keahlian atau pengalaman dalam bidang geoscience, geology, atau CFD.

Pengalaman dalam HPC dan Pemrograman Paralel:

Perusahaan: "experience high performance computing (hpc) parallel program".
John Smith: Tidak ada indikasi pengalaman dalam HPC atau pemrograman paralel.

Tidak Cocok: John Smith tidak menunjukkan pengalaman dalam HPC atau pemrograman paralel.

Kemampuan Pemecahan Masalah dan Penulisan Laporan Teknis:

Perusahaan: "excellent problem solve skill", "qualification write technical report journal article".

John Smith: "strong problem-solving skill", namun tidak ada bukti spesifik tentang penulisan laporan teknis atau artikel jurnal.

Sebagian Cocok: John Smith menunjukkan kemampuan pemecahan masalah yang kuat, tetapi tidak ada informasi tentang penulisan laporan teknis atau artikel jurnal.

Kesimpulan:

Dari kedua deskripsi pekerjaan, kalimat pertama "glove-wiki-gigaword-50"(Agile Data Pro) lebih cocok untuk John Smith dalam hal keahlian teknis dan pengalaman dalam analisis data dan pembelajaran mesin. Namun, karena posisi ini adalah magang yang tidak dibayar dan ditujukan untuk mahasiswa sarjana, itu tidak sesuai dengan tingkat pengalaman dan aspirasi karier John Smith.

Kalimat kedua (GI Geoscience) tidak cocok untuk John Smith karena membutuhkan keahlian dan pengalaman dalam bidang geoscience, geology, CFD, dan HPC, yang tidak dimiliki oleh John Smith.

Kesimpulan final:

Meskipun "glove-wiki-gigaword-50" punya skor 0.972 - 0.972 dalam 20 besar , tapi kurang tepat dalam memprediksi pekerjaan ketimbang model "glove-wiki-gigaword-300" yang hanya punya skor 0.921 - 0.935 dalam 20 besar

### model glove-wiki-gigaword-100

In [ ]:
glove_vectors = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 128.1/128.1MB downloaded


In [ ]:
input_text = "At the age of 28, I, John Smith, am an enthusiastic and highly skilled data analyst seeking to advance my career by applying for a position in the data field. With over five years of experience in data analysis and a proven track record of successfully implementing data-driven strategies, I have honed my expertise in various analytical tools and methodologies. I hold a Bachelor's degree in Computer Science from the University of California, Berkeley, and a Master's degree in Data Science from Stanford University. Throughout my career, I have demonstrated exceptional analytical capabilities, strong problem-solving skills, and a deep understanding of statistical models and machine learning algorithms. My passion for uncovering insights from complex datasets and my commitment to continuous learning make me a valuable candidate for any data-driven organization."
input_text = input_text.lower()
input_text = re.sub(r'[^\w\s]', '', input_text)
tokens = word_tokenize(input_text)
stop_words = set(stopwords.words('english'))
tokens = [word for word in tokens if word not in stop_words]
lemmatizer = WordNetLemmatizer()
tokens = [lemmatizer.lemmatize(word) for word in tokens]
input_text = ' '.join(tokens)

In [ ]:
def get_average_glove(tokens, model, num_features):
    valid_words = [word for word in tokens if word in model]
    if not valid_words:
        return np.zeros(num_features)
    return np.mean([model[word] for word in valid_words], axis=0)

dataset = pd.read_csv(output)

corpus_tokens = [word_tokenize(re.sub(r'[^\w\s]', '', doc.lower())) for doc in dataset["deskripsi_keterampilan_final"]]
corpus_tokens = [[lemmatizer.lemmatize(word) for word in tokens if word not in stop_words] for tokens in corpus_tokens]
num_features = 100
corpus_embeddings = np.array([get_average_glove(tokens, glove_vectors, num_features) for tokens in corpus_tokens])

input_tokens = word_tokenize(input_text)
input_embedding = get_average_glove(input_tokens, glove_vectors, num_features)

In [ ]:
cosine_similarities = cosine_similarity([input_embedding], corpus_embeddings).flatten()
dataset["cosine_similarity"] = cosine_similarities

In [ ]:
top_similar = dataset.sort_values(by='cosine_similarity', ascending=False).head(20)
top_similar[["skill","job title","deskripsi_keterampilan_final","cosine_similarity"]]

,skill,job title,deskripsi_keterampilan_final,cosine_similarity
13111,Information Technology,data science artificial intelligence machine learning internship,company agile data pro digital lab provider data science solution harness power machine learning generative ai integrated platform help capture data multiple source analyze use create model predict business issue outcomes team phd hold multiple patent leverage latest technology deliver reliable efficient solution value customer job role data science ai ml internship unpaid location onsite san jose ca pm min day week unpaid internship extraordinary opportunity learn experienced professional provide mentorship guidance throughout internship internship gain valuable experience artificial intelligence ai machine leaning ml natural language processing nlp teamwork essential skill successful career field suggestive project base industry real life data develop gcp azure cloud also give chance practice work data scientist real life work datasets seek highly motivate analytical individual join team intern intern opportunity work project gain hand experience data analysis machine learning experimentation statistical modeling collaborate team data scientist help drive business growth requirement familiarity programming language python prefer java c basic understanding sql ability identify interpret domain heavily nuanced data able work multiple project meet quality standard deadline ability work independently part team education qualification look highly talented motivate undergraduate student pursue degree computer science computer engineering related technical field interested pursue opportunity please send updated resume sam agile data pro com,0.965876
22397,Research,group summer research program intern advanced concept technology,advanced concept technology group look exceptional engineer scientist mathematician excite work challenge problem join team group develop technology protect deployed sailor soldier asset advanced missile threat enable operation rapidly evolve complex electronic warfare ew environment group research development activity include concept design advance algorithm development software hardware prototyping experimental field testing data analysis team focus create new capability rapid prototyping software hardware particular interest group development advanced electronic warfare system prototype advance sensor prototype novel signal processing technique resource allocation mathematical optimization approach artificial intelligence enable development next generation system fleet surface asset defense due wide ranging nature research dynamic group comprise diverse team scientist engineer expertise physic mathematics computer science various engineering background group value inclusiveness foster mentor level promote critical innovative thinking address need nation candidate part highly collaborative team thrive solve complex challenge foster innovation support career growth dedicate make positive impact nation job description summer research program intern need investigate design algorithm area classical machine learning bayesian inference work involve explore implementing compare performance use data simulation well measure data candidate comfortable read academic paper turn material high quality software tool current enrollment program working towards b phd degree mathematics computer science electrical engineering demonstrate effective organizational skill ability work independently team environment demonstrate strong write oral communication skills strong foundation probability linear algebra work knowledge statistical inference digital signal processing information theory prefer python julia programming experience strongly desire select candidate subject pre employment background investigation must able obtain maintain secret level dod security clearance mit lincoln laboratory equal employment opportunity eeo employer qualify applicant receive consideratio

# Hyper parameter tunning "glove-wiki-gigaword-300"

Agregasi embeding "Mean" polling

In [ ]:
glove_vectors = api.load("glove-wiki-gigaword-300")

In [ ]:
def get_average_glove(tokens, model, num_features, aggregation='mean'):
    valid_words = [word for word in tokens if word in model]
    if not valid_words:
        return np.zeros(num_features)

    if aggregation == 'mean':
        return np.mean([model[word] for word in valid_words], axis=0)
    elif aggregation == 'max':
        return np.max([model[word] for word in valid_words], axis=0)
    elif aggregation == 'sum':
        return np.sum([model[word] for word in valid_words], axis=0)
    else:
        raise ValueError("Unsupported aggregation method: {}".format(aggregation))

In [ ]:
def preprocess_text(text, remove_stopwords=True, lemmatize=True):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    if remove_stopwords:
        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if word not in stop_words]
    if lemmatize:
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

In [ ]:
input_text = "At the age of 28, I, John Smith, am an enthusiastic and highly skilled data analyst seeking to advance my career by applying for a position in the data field. With over five years of experience in data analysis and a proven track record of successfully implementing data-driven strategies, I have honed my expertise in various analytical tools and methodologies. I hold a Bachelor's degree in Computer Science from the University of California, Berkeley, and a Master's degree in Data Science from Stanford University. Throughout my career, I have demonstrated exceptional analytical capabilities, strong problem-solving skills, and a deep understanding of statistical models and machine learning algorithms. My passion for uncovering insights from complex datasets and my commitment to continuous learning make me a valuable candidate for any data-driven organization."
input_tokens = preprocess_text(input_text)
input_embedding = get_average_glove(input_tokens, glove_vectors, glove_vectors.vector_size, aggregation='mean')

In [ ]:
dataset = pd.read_csv(output)

corpus_tokens = [preprocess_text(doc) for doc in dataset["deskripsi_keterampilan_final"]]
corpus_embeddings = np.array([get_average_glove(tokens, glove_vectors, glove_vectors.vector_size, aggregation='mean') for tokens in corpus_tokens])

In [ ]:
cosine_similarities = cosine_similarity([input_embedding], corpus_embeddings).flatten()
dataset["cosine_similarity"] = cosine_similarities

In [ ]:
top_similar = dataset.sort_values(by='cosine_similarity', ascending=False).head(10)
top_similar[["skill","job title","deskripsi_keterampilan_final","cosine_similarity"]]

skill  \
13111  Information Technology   
9203                    Other   
22397                Research   
19613                 Analyst   
29493               Education   
4277                  Science   
4486                  Science   
19252  Information Technology   
1784       Project Management   
19401             Engineering   

                                                                       job title  \
13111           data science artificial intelligence machine learning internship   
9203   senior bioinformatics research scientist bioinformatic research scientist   
22397           group summer research program intern advanced concept technology   
19613                                                             data scientist   
29493                                    researcher doctor of philosophy student   
4277                                                       senior data scientist   
4486                                                       senior data scientist   
19252                                                             data scientist   
1784                                                              senior manager   
19401                                     research development civil engineering   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

dengan agregasi embeding mean..skor 20 besarnya adalah 0.927 - 0.935

yang sebelumnya skornya adalah kisaran 0.921 - 0.935, pada dasrnya sama karena agregasi embedingnya kurang lebih mirip dengan contoh ori

### Agregasi embeding sum polling

In [ ]:
glove_vectors = api.load("glove-wiki-gigaword-300")

def get_average_glove(tokens, model, num_features, aggregation='mean'):
    valid_words = [word for word in tokens if word in model]
    if not valid_words:
        return np.zeros(num_features)

    if aggregation == 'mean':
        return np.mean([model[word] for word in valid_words], axis=0)
    elif aggregation == 'max':
        return np.max([model[word] for word in valid_words], axis=0)
    elif aggregation == 'sum':
        return np.sum([model[word] for word in valid_words], axis=0)
    else:
        raise ValueError("Unsupported aggregation method: {}".format(aggregation))

def preprocess_text(text, remove_stopwords=True, lemmatize=True):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    if remove_stopwords:
        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if word not in stop_words]
    if lemmatize:
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens


input_text = "At the age of 28, I, John Smith, am an enthusiastic and highly skilled data analyst seeking to advance my career by applying for a position in the data field. With over five years of experience in data analysis and a proven track record of successfully implementing data-driven strategies, I have honed my expertise in various analytical tools and methodologies. I hold a Bachelor's degree in Computer Science from the University of California, Berkeley, and a Master's degree in Data Science from Stanford University. Throughout my career, I have demonstrated exceptional analytical capabilities, strong problem-solving skills, and a deep understanding of statistical models and machine learning algorithms. My passion for uncovering insights from complex datasets and my commitment to continuous learning make me a valuable candidate for any data-driven organization."
input_tokens = preprocess_text(input_text)
input_embedding = get_average_glove(input_tokens, glove_vectors, glove_vectors.vector_size, aggregation='sum')

dataset = pd.read_csv(output)

corpus_tokens = [preprocess_text(doc) for doc in dataset["deskripsi_keterampilan_final"]]
corpus_embeddings = np.array([get_average_glove(tokens, glove_vectors, glove_vectors.vector_size, aggregation='sum') for tokens in corpus_tokens])


cosine_similarities = cosine_similarity([input_embedding], corpus_embeddings).flatten()
dataset["cosine_similarity"] = cosine_similarities

In [ ]:
top_similar = dataset.sort_values(by='cosine_similarity', ascending=False).head(10)
top_similar[["skill", "job title", "deskripsi_keterampilan_final", "cosine_similarity"]]

skill  \
13111  Information Technology   
9203                    Other   
22397                Research   
19613                 Analyst   
29493               Education   
4277                  Science   
4486                  Science   
19252  Information Technology   
1784       Project Management   
19401             Engineering   

                                                                       job title  \
13111           data science artificial intelligence machine learning internship   
9203   senior bioinformatics research scientist bioinformatic research scientist   
22397           group summer research program intern advanced concept technology   
19613                                                             data scientist   
29493                                    researcher doctor of philosophy student   
4277                                                       senior data scientist   
4486                                                       senior data scientist   
19252                                                             data scientist   
1784                                                              senior manager   
19401                                     research development civil engineering   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               